## Notebook Summary
#### *Capstone: Modeling*
---
This contents of this notebook includes the modeling for both targets. It also includes reflections on model performance.

In [1]:
# import packages
import pandas as pd
import numpy as np
import re
import pickle
from sklearn.compose import make_column_transformer
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.linear_model import LinearRegression, ElasticNet
from sklearn import metrics
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_squared_error
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.decomposition import PCA
from sklearn.metrics import r2_score
from sklearn.neighbors import KNeighborsRegressor
import math
import sklearn

pd.set_option('display.max_rows', None)
# pd.set_option('max_colwidth', 150)

# Set a random seed.
np.random.seed(17)

import warnings
warnings.filterwarnings('ignore')

# Modeling 1
---

In [2]:
#read in modeling datasets
X_train = pd.read_csv('../Capstone/cleaned_datasets/modeling/model_1/X_train_final.csv')
X_test = pd.read_csv('../Capstone/cleaned_datasets/modeling/model_1/X_test_final.csv')
y_train = pd.read_csv('../Capstone/cleaned_datasets/modeling//model_1/y_train.csv')
y_test = pd.read_csv('../Capstone/cleaned_datasets/modeling/model_1/y_test.csv')

In [3]:
y_train.drop(columns=['rcdts', 'Unnamed: 0'], inplace=True)
y_test.drop(columns=['rcdts', 'Unnamed: 0'], inplace=True)
X_train.drop(columns='Unnamed: 0', inplace=True)
X_test.drop(columns='Unnamed: 0', inplace=True)

### Baseline

R-squared measures the strength of the relationship between your model and the dependent variable on a convenient 0 – 100% scale

In [4]:
# baseline
y2 = np.full((len(y_test),), y_test.mean())
mse = mean_squared_error(y_test, y2, squared=False)
r2 = metrics.r2_score(y_test, y2)
print('LR rmse:', mse)
print('r2:', r2)

LR rmse: 104.10564288494938
r2: 0.0


In [5]:
#create a dummy regressor
dummy_reg = DummyRegressor(strategy='mean')

#fit on training
dummy_reg.fit(X_train, y_train)

# pred on test
y_pred = dummy_reg.predict(X_test)

# baseline
mse = mean_squared_error(y_test, y_pred, squared=False)
print('LR rmse:', mse)

LR rmse: 104.10564683878363


### Linear Regression

In [6]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [7]:
scores = cross_val_score(estimator = lr, X=X_train, y=y_train, cv = 5)

In [8]:
print('R2 train:', lr.score(X_train, y_train))
print('R2 test:', lr.score(X_test, y_test))
print('rmse:', mean_squared_error(y_test,lr.predict(X_test), squared=False))
print('cross val score:', scores.mean())

R2 train: 0.3614004718901326
R2 test: 0.3281010568101229
rmse: 85.33481462749047
cross val score: 0.3201877630571496


In [9]:
lr_features = pd.DataFrame(zip(X_train.columns, lr.coef_[0])).sort_values(by=1, ascending=False)
lr_features.rename(columns={1: 'coef', 0:'feature'}, inplace=True)
lr_features['abs'] = [abs(v) for v in lr_features['coef']]

#### List of 20 predictive features - Linear Regression
A positive coefficient indicates that as the value of the independent variable increases, the mean of the dependent variable also tends to increase. A negative coefficient suggests that as the independent variable increases, the dependent variable tends to decrease.

In [10]:
lr_features.sort_values(by='abs', ascending=False).head(10)

,feature,coef,abs
140,fte,28.745833,28.745833
143,chronically_truant_students,22.063626,22.063626
106,county_Winnebago,18.629204,18.629204
128,chronic_absenteeism_white,14.129990,14.129990
117,%_student_enrollment_el,-14.052626,14.052626
94,county_Stephenson,10.949398,10.949398
138,summative_designation,-10.291834,10.291834
144,student_attendance_rate,-9.706409,9.706409
132,chronic_absenteeism_two_or_more_races,7.089313,7.089313
0,region_type_City - Small/Mid-Size,6.666493,6.666493


### ElasticNet

In [11]:
enet = ElasticNet()
enet.fit(X_train, y_train)

ElasticNet()

In [12]:
print('R2 train:', enet.score(X_train, y_train))
print('R2 test:', enet.score(X_test, y_test))
print('rmse:', mean_squared_error(y_test,enet.predict(X_test), squared=False))

R2 train: 0.32567144408321613
R2 test: 0.302547787492492
rmse: 86.94237478939543


In [13]:
enet = ElasticNet()

enet_params = {
    'l1_ratio': [0.75,0.85,0.9],
    'alpha': [0.0001, 0.001, 0.01, 0.1, 1, 10],

}

gs_enet = GridSearchCV(
    estimator = enet,
    param_grid = enet_params,
    n_jobs=-1,
    verbose=True
)

In [14]:
gs_enet.fit(X_train, y_train)

Fitting 5 folds for each of 18 candidates, totalling 90 fits


GridSearchCV(estimator=ElasticNet(), n_jobs=-1,
             param_grid={'alpha': [0.0001, 0.001, 0.01, 0.1, 1, 10],
                         'l1_ratio': [0.75, 0.85, 0.9]},
             verbose=True)

In [15]:
print('R2 train:', gs_enet.score(X_train, y_train))
print('R2 test:', gs_enet.score(X_test, y_test))
print('rmse:', mean_squared_error(y_test, gs_enet.predict(X_test), squared=False))
print('cross val score:', gs_enet.best_score_)
print('best params:', gs_enet.best_params_)

R2 train: 0.3608826606066132
R2 test: 0.32873761565191606
rmse: 85.29438183377184
cross val score: 0.32296420811456533
best params: {'alpha': 0.1, 'l1_ratio': 0.75}


### Random Forest

In [16]:
forest = RandomForestRegressor()
forest.fit(X_train, y_train.values.ravel())

RandomForestRegressor()

In [17]:
print('R2 train:', forest.score(X_train, y_train))
print('R2 test:', forest.score(X_test, y_test))
print('rmse:', mean_squared_error(y_test, forest.predict(X_test), squared=False))

R2 train: 0.9321054530316758
R2 test: 0.5583533861716778
rmse: 69.18496457461222


### Bayes Search + RandomForestRegressor

In [18]:
forest = RandomForestRegressor()

pipe_forest_params = {
    'max_depth': Integer(1, 3),
    'n_estimators': Integer(50,200),
    'max_features': ['sqrt', 'log2', None]
    } 

bs_forest=BayesSearchCV(
    estimator=forest,
    search_spaces=pipe_forest_params,
    n_iter=50,
    verbose=1,
    cv=5,
    n_jobs=-1
)

In [19]:
# bs_forest.fit(X_train, y_train.values.ravel())

In [20]:
#to pickle
# pickle.dump(bs_forest, open('../Capstone/pickles/bs_forest.pkl', 'wb'))

#load in pickled model
bs_forest = pickle.load(open('../Capstone/pickles/bs_forest.pkl','rb'))

In [21]:
print('R2 train:', bs_forest.score(X_train,y_train))
print('R2 test:', bs_forest.score(X_test,y_test))
print('rmse:', mean_squared_error(y_test, bs_forest.predict(X_test), squared=False))
print('cross val score:', bs_forest.best_score_)

R2 train: 0.5380622352724596
R2 test: 0.3782998461470026
rmse: 82.08518056053862
cross val score: 0.3676312435300403


In [22]:
bs_forest.best_estimator_

RandomForestRegressor(max_depth=3, max_features=None, n_estimators=200)

### GridSearch + RandomForestRegressor

In [23]:
rf = RandomForestRegressor(max_features=None)

gs_params = {
    'max_depth': [2,3,4],
    'n_estimators': [55,60,65],
    'min_samples_split': [2,3],
    'min_samples_leaf': [1,2,3]

}

gs_rf = GridSearchCV(
    estimator = rf,
    param_grid = gs_params,
    n_jobs=-1,
    verbose=True
)

In [24]:
# gs_rf.fit(X_train, y_train.values.ravel())

In [25]:
# #to pickle
# pickle.dump(gs_rf, open('../Capstone/pickles/gs_forest.pkl', 'wb'))

# # #load in pickled model
gs_rf = pickle.load(open('../Capstone/pickles/gs_forest.pkl','rb'))

In [26]:
print('R2 train:', gs_rf.score(X_train,y_train))
print('R2 test:', gs_rf.score(X_test,y_test))
print('rmse:', mean_squared_error(y_test, gs_rf.predict(X_test), squared=False))
print('cross val score:', gs_rf.best_score_)
print('best params:', gs_rf.best_params_)

R2 train: 0.6642229585089916
R2 test: 0.43511417022091503
rmse: 78.24464251530571
cross val score: 0.4507877122298655
best params: {'max_depth': 4, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 65}


### XGBoost Regressor + GridSearchCV

In [27]:
xgboost1 = xgb.XGBRegressor()
xgboost1.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
             grow_policy='depthwise', importance_type=None,
             interaction_constraints='', learning_rate=0.300000012, max_bin=256,
             max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
             max_depth=6, max_leaves=0, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, ...)

In [28]:
print('R2 train:', xgboost1.score(X_train, y_train))
print('R2 test:', xgboost1.score(X_test, y_test))
print('RMSE', mean_squared_error(y_test, xgboost1.predict(X_test), squared=False))

R2 train: 0.9888995877151291
R2 test: 0.5227867939272758
RMSE 71.91682356852033


In [29]:
#gridsearch
xgboost= xgb.XGBRegressor()

xg_params = {
    'max_depth': [2,3],
    'n_estimators': [50,75,100],
    'subsample': [0.4, 0.5, 0.6]

}   

gs_xg = GridSearchCV(
    estimator = xgboost,
    param_grid = xg_params,
    n_jobs=-1,
    verbose=True
)

In [30]:
# gs_xg.fit(X_train, y_train)

In [31]:
#to pickle
# pickle.dump(gs_xg, open('../Capstone/pickles/gs_xg.pkl', 'wb'))

# load in pickled model
gs_xg = pickle.load(open('../Capstone/pickles/gs_xg.pkl','rb'))

In [32]:
print('R2 train:', gs_xg.score(X_train, y_train))
print('R2 test', gs_xg.score(X_test, y_test))
print('RMSE:', mean_squared_error(y_test, gs_xg.predict(X_test), squared=False))
print('cross val score:', gs_xg.best_score_)
print(gs_xg.best_params_)

R2 train: 0.8585159977736353
R2 test 0.5355641021690795
RMSE: 70.94751044714444
cross val score: 0.5538093577396694
{'max_depth': 3, 'n_estimators': 75, 'subsample': 0.6}


In [33]:
# xgboost_feats = pd.DataFrame(zip(X_train.columns, gs_xg.best_estimator_.feature_importances_))
# xgboost_feats.rename(columns={0: 'features', 1: 'feature importance'}, inplace=True)
# xgboost_feats.sort_values(by='feature importance', ascending=False)

#### Due to feature importances here being biased towards continuous variables, I chose to look at permutation feature importance

In [34]:
from sklearn.inspection import permutation_importance
perm_importance = permutation_importance(gs_xg, X_test, y_test)

In [35]:
# return df of permutation importances
xgb_permfeat = pd.DataFrame(zip(X_test.columns, perm_importance['importances_mean'])).sort_values(by=1, ascending=False)
xgb_permfeat.rename(columns={0: 'features', 1: 'mean perm feature imp'}, inplace=True)

# The permutation feature importance is defined to be the decrease in a model score when a single feature value is randomly shuffled
# negative score means the preds on shuffled data is more acc on shuffled data, which means feat does not contribute much to predictions
# the drop in the model score is indicative of how much the model depends on the feature
# doesn't show +/-, but what feats are most important to the model

In [36]:
xgb_permfeat[:20]

,features,mean perm feature imp
140,fte,0.338743
106,county_Winnebago,0.090211
143,chronically_truant_students,0.068733
144,student_attendance_rate,0.035269
136,%_math_proficiency_low_income,0.033615
116,%_student_enrollment_two_or_more_races,0.031921
94,county_Stephenson,0.017892
3,region_type_Town,0.015776
146,$_instructional_expenditure_per_pupil,0.013509
117,%_student_enrollment_el,0.012339


### KNNRegressor

In [37]:
neigh = KNeighborsRegressor(n_neighbors=7, algorithm ='ball_tree')
neigh.fit(X_train, y_train)

KNeighborsRegressor(algorithm='ball_tree', n_neighbors=7)

In [38]:
print(neigh.score(X_train, y_train))
print(neigh.score(X_test, y_test))
print(mean_squared_error(y_test, neigh.predict(X_test), squared=False))

0.5598368220135573
0.4275996218598098
78.76335851060024


In [39]:
#gridsearch
knn = KNeighborsRegressor()

knn_params = {
    'n_neighbors': [3,4,5,6,7],
    'leaf_size': [2,3,4,5],
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'weights': ['uniform', 'distance']
}   

gs_knn = GridSearchCV(
    estimator = knn,
    param_grid = knn_params,
    n_jobs=-1,
    verbose=True
)

In [40]:
# commenting out to not rerun again
# gs_knn.fit(X_train, y_train)

In [41]:
#to pickle
# pickle.dump(gs_knn, open('../Capstone/pickles/gs_knn.pkl', 'wb'))

#load in pickled model
gs_knn = pickle.load(open('../Capstone/pickles/gs_knn.pkl','rb'))

In [42]:
print('r2 train:', gs_knn.score(X_train, y_train))
print('r2 test:', gs_knn.score(X_test, y_test))
print('rmse:',mean_squared_error(y_test, gs_knn.predict(X_test), squared=False))
print('cross val score:', gs_knn.best_score_)
print('best params:', gs_knn.best_params_)

r2 train: 0.9999999999999959
r2 test: 0.46599946127840575
rmse: 76.07555300382253
cross val score: 0.4269542655043252
best params: {'algorithm': 'auto', 'leaf_size': 2, 'n_neighbors': 4, 'weights': 'distance'}


### ExtraTrees Model

In [43]:
extrees = ExtraTreesRegressor()
extrees.fit(X_train, y_train)

ExtraTreesRegressor()

In [44]:
print(extrees.score(X_train, y_train))
print(extrees.score(X_test, y_test))
print(mean_squared_error(y_test, extrees.predict(X_test), squared=False))

1.0
0.5946613046504314
66.28012259852575


In [45]:
#gridsearch
etrees = ExtraTreesRegressor()

et_params = {
    'n_estimators': [50,75,100],
    'max_depth': [2,3,4],
    'min_samples_split': [2,3]
}   

gs_et = GridSearchCV(
    estimator = etrees,
    param_grid = et_params,
    n_jobs=-1,
    verbose=True
)

In [46]:
#fit on X_train
gs_et.fit(X_train, y_train.values.ravel())

Fitting 5 folds for each of 18 candidates, totalling 90 fits


GridSearchCV(estimator=ExtraTreesRegressor(), n_jobs=-1,
             param_grid={'max_depth': [2, 3, 4], 'min_samples_split': [2, 3],
                         'n_estimators': [50, 75, 100]},
             verbose=True)

In [47]:
print('R2 train:', gs_et.score(X_train, y_train))
print('R2 test:', gs_et.score(X_test, y_test))
print('rmse:', mean_squared_error(y_test, gs_et.predict(X_test), squared=False))
print('cross val score:', gs_et.best_score_)
print('best params:', gs_et.best_params_)

R2 train: 0.5934177652305626
R2 test: 0.4098449760969738
rmse: 79.97556643253859
cross val score: 0.4399147957269699
best params: {'max_depth': 4, 'min_samples_split': 3, 'n_estimators': 100}


# Model #2 w/ Some Additional Feature Engineering
---
After inspecting the permutation feature importances, I am choosing to do some additional feature engineering for comparison. 
- not dropping collinear features during pre-processing since I'm using the XGBoost model
- dropping dummied county columns
- adjusting target to be incidents per pupil

In [48]:
X_train_adj = pd.read_csv('../Capstone/cleaned_datasets/modeling/model_2/X_train_adj.csv')
y_train_adj = pd.read_csv('../Capstone/cleaned_datasets/modeling/model_2/y_train_adj.csv')
X_test_adj = pd.read_csv('../Capstone/cleaned_datasets/modeling/model_2/X_test_adj.csv')
y_test_adj = pd.read_csv('../Capstone/cleaned_datasets/modeling/model_2/y_test_adj.csv')

#drop Unnamed cols & rcdts
X_train_adj.drop(columns='Unnamed: 0', inplace=True)
X_test_adj.drop(columns='Unnamed: 0', inplace=True)
y_train_adj.drop(columns=['Unnamed: 0', 'rcdts'], inplace=True)
y_test_adj.drop(columns=['Unnamed: 0', 'rcdts'], inplace=True)

In [49]:
# baseline
y2 = np.full((len(y_test_adj),), y_test_adj.mean())
mse = mean_squared_error(y_test_adj, y2, squared=False)
r2 = metrics.r2_score(y_test_adj, y2)
print('LR rmse:', mse)
print('r2:', r2)

LR rmse: 0.16731867065354478
r2: 0.0


### Linear Regression

In [50]:
lr=LinearRegression()
lr.fit(X_train_adj, y_train_adj)

LinearRegression()

In [51]:
print('R2 train:', lr.score(X_train_adj, y_train_adj))
print('R2 test:', lr.score(X_test_adj, y_test_adj))
print('RMSE:', mean_squared_error(y_test_adj, lr.predict(X_test_adj), squared=False))
print('cross val score:', cross_val_score(estimator = lr, X=X_train_adj, y=y_train_adj, cv = 5).mean())

R2 train: 0.2779397546689447
R2 test: 0.25044299415005644
RMSE: 0.14485941910207298
cross val score: 0.2379540288432363


## XGBoost

In [52]:
xgboost2 = XGBRegressor()
xgboost2.fit(X_train_adj, y_train_adj)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
             grow_policy='depthwise', importance_type=None,
             interaction_constraints='', learning_rate=0.300000012, max_bin=256,
             max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
             max_depth=6, max_leaves=0, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, ...)

In [53]:
print('R2 train:', xgboost2.score(X_train_adj, y_train_adj))
print('RMSE:', mean_squared_error(y_test_adj, xgboost2.predict(X_test_adj), squared=False))
print('cross val score:', cross_val_score(estimator = xgboost2, X=X_train_adj, y=y_train_adj, cv = 5).mean())

R2 train: 0.9874240756451862
RMSE: 0.15907344708091317
cross val score: 0.22673209420976637


In [54]:
#gridsearch
xgboost2= xgb.XGBRegressor()

xg_params = {
    'max_depth': [2,3,4],
    'n_estimators': [100,200, 225]
    # 'subsample': [0.6, 0.7, 0.8,0.9]

}   

gs_xg2 = GridSearchCV(
    estimator = xgboost2,
    param_grid = xg_params,
    n_jobs=-1,
    verbose=True
)

In [55]:
# gs_xg2.fit(X_train_adj, y_train_adj)

In [56]:
# to pickle
# pickle.dump(gs_xg2, open('../Capstone/pickles/gs_xg2.pkl', 'wb'))

# load in pickled model
gs_xg2 = pickle.load(open('../Capstone/pickles/gs_xg2.pkl','rb'))

In [57]:
print('R2 train:', gs_xg2.score(X_train_adj, y_train_adj))
print('R2 test:', gs_xg2.score(X_test_adj, y_test_adj))
print('RMSE:', mean_squared_error(y_test_adj, gs_xg2.predict(X_test_adj), squared=False))
print('cross val score:', gs_xg2.best_score_)
print('best params:', gs_xg2.best_params_)

R2 train: 0.9689761506289547
R2 test: 0.1523136891953515
RMSE: 0.15405010205817188
cross val score: 0.300552705787377
best params: {'max_depth': 4, 'n_estimators': 200}


In [58]:
#calculate permutation feature importance
perm_importance2 = permutation_importance(gs_xg2, X_train_adj, y_train_adj)

In [59]:
# return df of permutation importances
xgb2_permfeat2 = pd.DataFrame(zip(X_train_adj.columns, perm_importance2['importances_mean'])).sort_values(by=1, ascending=False)
xgb2_permfeat2.rename(columns={0: 'features', 1: 'mean perm feature imp'}, inplace=True)
xgb2_permfeat2

,features,mean perm feature imp
40,%_math_proficiency_low_income,0.134403
14,%_student_enrollment_black_or_african_american,0.127834
48,chronically_truant_students,0.107343
50,student_attendance_rate,0.096260
13,%_student_enrollment_white,0.085477
47,teacher_retention_rate,0.080803
42,%_math_proficiency,0.077035
20,%_student_enrollment_el,0.070165
49,student_chronic_truancy_rate,0.060252
54,%_local_property_taxes,0.055264


#### Normalizing Target
I noticed that the target was skewed right which could be impacting model performance. I am using strategies to normalize the target. I am only using this with a basic linear regresssion and XGBoost to do time constraints.

In [60]:
from sklearn.preprocessing import QuantileTransformer
from sklearn.compose import TransformedTargetRegressor

In [61]:
#XGBRegressor
transformer = QuantileTransformer(output_distribution='normal')
regressor = XGBRegressor()
regr = TransformedTargetRegressor(regressor=regressor, transformer=transformer)

In [62]:
regr.fit(X_train_adj, y_train_adj)

TransformedTargetRegressor(regressor=XGBRegressor(base_score=None, booster=None,
                                                  callbacks=None,
                                                  colsample_bylevel=None,
                                                  colsample_bynode=None,
                                                  colsample_bytree=None,
                                                  early_stopping_rounds=None,
                                                  enable_categorical=False,
                                                  eval_metric=None,
                                                  feature_types=None,
                                                  gamma=None, gpu_id=None,
                                                  grow_policy=None,
                                                  importance_type=None,
                                                  interaction_constraints=None,
                                                  learning_rate=None,
                                                  max_bin=None,
                                                  max_cat_threshold=None,
                                                  max_cat_to_onehot=None,
                                                  max_delta_step=None,
                                                  max_depth=None,
                                                  max_leaves=None,
                                                  min_child_weight=None,
                                                  missing=nan,
                                                  monotone_constraints=None,
                                                  n_estimators=100, n_jobs=None,
                                                  num_parallel_tree=None,
                                                  predictor=None,
                                                  random_state=None, ...),
                           transformer=QuantileTransformer(output_distribution='normal'))

In [63]:
print(regr.score(X_train_adj, y_train_adj))
print(cross_val_score(estimator = regr, X=X_train_adj, y=y_train_adj, cv = 5).mean())
print('RMSE:', mean_squared_error(y_test_adj, regr.predict(X_test_adj), squared=False))

0.7752805578911698
0.10152603103005202
RMSE: 0.14747902347359107


In [64]:
#LinearRegression
transformer = QuantileTransformer(output_distribution='normal')
regressor = LinearRegression()
regr = TransformedTargetRegressor(regressor=regressor, transformer=transformer)

In [65]:
regr.fit(X_train_adj, y_train_adj)

TransformedTargetRegressor(regressor=LinearRegression(),
                           transformer=QuantileTransformer(output_distribution='normal'))

In [66]:
print(regr.score(X_train_adj, y_train_adj))
print(cross_val_score(estimator = regr, X=X_train_adj, y=y_train_adj, cv = 5).mean())
print('RMSE:', mean_squared_error(y_test_adj, regr.predict(X_test_adj), squared=False))

-0.053256338399600534
-0.12008686908650287
RMSE: 0.17942941038909982


## Model Evaluation & Reflections
---

Looking across all of the models I tested, the XGBoost Regressor was the best model. I utilized both GridSearchCV and BayesSearchCV to hyperparameter tune. I used the rmse and cross-validates scores to evaluate the model. Both models performed better than baseline. The models I chose were overfit to the training datasets, but I was still able to gain valuable information about the features in relation to the targets. Because the feature importance values for the XGBoost model are typically biased to continuous datasets, I interpreted coefficients using the permutation feature importance. As a next step, I plan to do additional feature engineering and selection, normalize the target, and try applying polynomial features, PCA, and neural net models.